<a href="https://colab.research.google.com/github/xooca/simpletorch/blob/master/innoplexus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
Hu!pip install overrides
!pip install allennlp


In [0]:
from pathlib import Path
from typing import *
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util
from nltk.tokenize import TreebankWordTokenizer, word_tokenize, sent_tokenize
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score
import string
import nltk
import re
from collections import Counter
import gensim
from allennlp.training.trainer import Trainer
from nltk.stem import WordNetLemmatizer
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.fields import TextField, MetadataField, ArrayField
from allennlp.common.checks import ConfigurationError
from allennlp.data.iterators import BucketIterator
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder
from allennlp.data.token_indexers.elmo_indexer import ELMoTokenCharactersIndexer
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import ElmoTokenEmbedder
from allennlp.modules.token_embedders.bert_token_embedder import PretrainedBertEmbedder
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.token_indexers import PretrainedBertIndexer
from allennlp.training.metrics import auc,categorical_accuracy,f1_measure
from allennlp.training.learning_rate_schedulers import learning_rate_scheduler ,CosineWithRestarts
from allennlp.training.momentum_schedulers import momentum_scheduler,inverted_triangular
from allennlp.training.learning_rate_schedulers import noam,NoamLR

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [0]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
sample = pd.read_csv('/content/gdrive/My Drive/Studies/innoplexus/sample_submission_i5xnIZD.csv')
test = pd.read_csv('/content/gdrive/My Drive/Studies/innoplexus/test_tOlRoBf.csv')
train = pd.read_csv('/content/gdrive/My Drive/Studies/innoplexus/train_F3WbcTw.csv')

In [0]:
train.shape

(5279, 4)

In [0]:
def train_validate_test_split(df, train_percent=.8, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    train_end = train_end +1
    validate_end = int(validate_percent * m) + train_end
    print(train_end)
    print(validate_end)
    train = df.ix[perm[:train_end]]
    validate = df.ix[perm[train_end:validate_end]]
    return train, validate

In [0]:
train , validate = train_validate_test_split(train, train_percent=.8, validate_percent=.2, seed=None)

4224
5279


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


In [0]:
train.shape,validate.shape

((4224, 4), (1055, 4))

In [0]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
validate = validate.reset_index(drop=True)

In [0]:
train['sentiment'].value_counts()

2    3073
1     668
0     483
Name: sentiment, dtype: int64

In [0]:
#test['sentiment'].value_counts()

In [0]:
validate['sentiment'].value_counts()

2    752
1    169
0    134
Name: sentiment, dtype: int64

In [0]:
test.head()

,unique_hash,text,drug
0,9e9a8166b84114aca147bf409f6f956635034c08,"256 (previously stable on natalizumab), with 5...",fingolimod
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,On fingolimod and have been since December 201...,fingolimod
2,50b6d851bcff4f35afe354937949e9948975adf7,Apparently it's shingles! :-/ I do have a few ...,humira
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,If the Docetaxel doing once a week x3 weeks th...,tagrisso
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,"CC, Stelara worked in a matter of days for me....",stelara


In [0]:
sample.head()

,unique_hash,sentiment
0,9e9a8166b84114aca147bf409f6f956635034c08,0
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,0
2,50b6d851bcff4f35afe354937949e9948975adf7,0
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,0
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,0


In [0]:
import spacy
print('spaCy Version: %s' % (spacy.__version__))
spacy_nlp = spacy.load('en')


def removestopwords_spacy(val):
    spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
    doc = spacy_nlp(val)
    tokens = [token.text for token in doc if not token.is_stop]
    return ' '.join(tokens)
  
def lemmatizer(val):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(val)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

def lemmatizer_spacy(val):
    doc = spacy_nlp(val)
    lemmatizer = WordNetLemmatizer()
    word_list = [token for token in doc] 
    lemmatized_output = ' '.join([w.lemma_ if w.lemma_ != '-PRON-' else w.lower_ for w in word_list])
    #lemmatized_output = ' '.join([token.lemma_ for token in word_list])
    return lemmatized_output

def removenewline(val):
    val = str(val).replace('\n', ' ')
    return val

def removepunc(val):
    translator = str.maketrans('', '', string.punctuation)
    return str(val).translate(translator)

def removestopwords_nltk(val):
    nltk_stopwords = nltk.corpus.stopwords.words('english')
    #stopwords_1= ['not','nor','no','against','before','after','down','again','out','more']
    #for w in stopwords_1:
    #    nltk_stopwords.remove(w)
    tokens = nltk.tokenize.word_tokenize(val)
    tokens = [token for token in tokens if not token in nltk_stopwords]
    return ' '.join(tokens)

def removenonenglish(val):
    printable = set(string.printable)
    return ''.join(list(filter(lambda x: x in printable, val)))

    
def clean_data_level1(reviews,trainmode=True):
    reviews['text_original'] = reviews['text']
    reviews["text"] = reviews['text'].str.replace('[^a-zA-Z]+',' ')
    reviews["text"] = reviews["text"].apply(removenewline)
    reviews["text"] = reviews["text"].apply(removenonenglish)
    #reviews["text"] = reviews["text"].apply(spellcorrect)
    reviews["text"] = reviews["text"].apply(removepunc)
    reviews["text"] = reviews["text"].apply(lemmatizer_spacy)
    #reviews["text"] = reviews["text"].apply(lemmatizer)
    reviews["text"] = reviews["text"].apply(removestopwords_nltk)
    #reviews["text"] = reviews["text"].apply(removestopwords_spacy)
    reviews["text"] = reviews["text"].apply(lambda x:x.lower())
    if trainmode:
      reviews = converttodummies(reviews,['sentiment'],threshold=200)
    reviews = reviews.reset_index(drop=True)
    return reviews

import sys
from collections import OrderedDict
from datetime import datetime
def converttodummies(tmpdf,cols,skipcols=[],threshold=100):
    #converttodummies_col = tmpdf.columns
    try:
        for col in cols:
            if col not in skipcols:
                if tmpdf[col].nunique() <= threshold:
                    dummy = pd.get_dummies(tmpdf[col])
                    dummy.columns = [col+'_'+str(x) for x in dummy.columns]
                    tmpdf = pd.concat([tmpdf, dummy], axis=1)
                    #tmpdf = tmpdf.drop(col,axis=1)
                else:
                    pass
        return tmpdf
    except:
        sys.exit('ERROR : ' + str(datetime.now()) + ' : ' + sys.exc_info()[1])
        
def nullfinder(df):
    for col in df:
        if sum(df[col].isnull()) > 0:
            print(f"For {col} number of nulls are {sum(df[col].isnull())}")

spaCy Version: 2.1.6


In [0]:
train = clean_data_level1(train,trainmode=True)
validate = clean_data_level1(validate,trainmode=True)
test = clean_data_level1(test,trainmode=False)


In [0]:

class SentiDatasetReader(DatasetReader):
    def __init__(self,token_indexers,max_seq_len,review_column,id_column,label_column,istesting,embeddingtype='bert'):
        super().__init__()
        #self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        #print(token_indexers)
        self.max_seq_len = max_seq_len
        self.review_column  = review_column
        self.id_column = id_column
        self.label_column = label_column
        #self.token_indexers = ELMoTokenCharactersIndexer() 
        self.token_indexer = token_indexers
        self.max_seq_len = max_seq_len
        self.testing = istesting
        if embeddingtype == 'elmo':
            self.tokenizer = self.elmo_tokenizer
        elif embeddingtype == 'bert':
            self.tokenizer = self.bert_tokenizer
        else:
            self.tokenizer = SingleIdTokenIndexer()
        self.token_indexers = {"tokens": self.token_indexer}
    
    def elmo_tokenizer(self,x):
        return [w.text for w in
                SpacyWordSplitter(language='en_core_web_sm', 
                                  pos_tags=False).split_words(x)[:self.max_seq_len]]
    
    def bert_tokenizer(self,x):
        #print(self.token_indexer)
        return self.token_indexer.wordpiece_tokenizer(x)[:self.max_seq_len - 2]
    @overrides
    def text_to_instance(self,tokens,id,labels):
        
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
         
        id_field = MetadataField(id)
        fields["id"] = id_field
        if labels is None:
            labels = np.zeros(len(label_cols))
        label_field = ArrayField(array=labels)
        fields["label"] = label_field
        return Instance(fields)
    
    @overrides
    def _read(self, file_path):
        df = pd.read_csv(file_path)
        if self.testing: 
            df = df.head(1000)
        for i, row in df.iterrows():
          yield self.text_to_instance(
              [Token(x) for x in self.tokenizer(str(row[self.review_column]))],
              str(row[self.id_column]), 
              row[self.label_column].values,
          )

            

In [0]:

class SentiDatasetReaderTest(DatasetReader):
    def __init__(self,token_indexers,max_seq_len,review_column,id_column,label_column,istesting,embeddingtype='bert'):
        super().__init__()
        #self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        #print(token_indexers)
        self.max_seq_len = max_seq_len
        self.review_column  = review_column
        self.id_column = id_column
        self.label_column = label_column
        #self.token_indexers = ELMoTokenCharactersIndexer() 
        self.token_indexer = token_indexers
        self.max_seq_len = max_seq_len
        self.testing = istesting
        if embeddingtype == 'elmo':
            self.tokenizer = self.elmo_tokenizer
        elif embeddingtype == 'bert':
            self.tokenizer = self.bert_tokenizer
        else:
            self.tokenizer = SingleIdTokenIndexer()
        self.token_indexers = {"tokens": self.token_indexer}
    
    def elmo_tokenizer(self,x):
        return [w.text for w in
                SpacyWordSplitter(language='en_core_web_sm', 
                                  pos_tags=False).split_words(x)[:self.max_seq_len]]
    
    def bert_tokenizer(self,x):
        #print(self.token_indexer)
        return self.token_indexer.wordpiece_tokenizer(x)[:self.max_seq_len - 2]
    @overrides
    def text_to_instance(self,tokens,id,labels):
        
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
         
        id_field = MetadataField(id)
        fields["id"] = id_field
        return Instance(fields)
    
    @overrides
    def _read(self, file_path):
        df = pd.read_csv(file_path)
        if self.testing: 
            df = df.head(1000)
        for i, row in df.iterrows():
          yield self.text_to_instance(
              [Token(x) for x in self.tokenizer(str(row[self.review_column]))],
              str(row[self.id_column]), 
          )
            

In [0]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)


In [0]:
train.head()

,unique_hash,text,drug,sentiment,text_original,sentiment_0,sentiment_1,sentiment_2
0,4b45464468211e25b0ba095b7d9beee2f011939a,jan pm momcareteam write mom diagnose stage ns...,opdivo,2,"On Jan 05, 2016 11:02 PM MomCareTeam wrote: My...",0,0,1
1,67a405445e3e2141895ca34318da9a02e9c2dde7,another reason win take ocrelizumab progressiv...,ocrelizumab,1,Another reason I won't be taking Ocrelizumab f...,0,1,0
2,5a99d2f82fdf8df4d715bb7e6620326a1ce8a112,peg brave try quite remedy already really good...,ocrevus,0,"Peg, you've been brave and tried quite a few r...",1,0,0
3,13f717b1740d24d4322ddf07f76e42e9f9b3f2cc,hello interesting appointment post ru cat wow ...,ocrevus,2,"Hello, Such an interesting appointment and pos...",0,0,1
4,20bf0873a730336db54199689605645c5f2dc6c3,confused ocrelizumab clone rituximab see many ...,ocrelizumab,2,"So, I am confused! Is Ocrelizumab a clone of R...",0,0,1


In [0]:
config = Config(
    istesting=False,
    seed=12,
    batch_size=32,
    #lr=3e-4,
    lr=0.01,
    epochs=80,
    hidden_size=32,
    max_seq_len=500, 
    max_vocab_size=10000000,
    review_column='text_original',
    id_column='unique_hash',
    label_column=['sentiment_0','sentiment_1','sentiment_2'],
    train_file_name = 'train.csv',
    test_file_name = 'test.csv',
    data_folder_loc = Path("./"),
    token_indexers = ELMoTokenCharactersIndexer(),
#   token_indexers = PretrainedBertIndexer(pretrained_model="bert-base-uncased",max_pieces=500,do_lowercase=True,),
    bidirectional=True, 
    batch_first=True,
    embeddingtype = 'elmo'
)


torch.manual_seed(config.seed)
USE_GPU = torch.cuda.is_available()

#dim = 
class BertSentencePooler(Seq2VecEncoder):
    def forward(self, embs,mask):
        return embs[:, 0]
    
    @overrides
    def get_output_dim(self):
        return bert_dim

class data_setup(SentiDatasetReader):
    def __init__(self,config):
        self.config = config
        super().__init__( token_indexers = self.config.token_indexers,
            max_seq_len = self.config.max_seq_len,
            review_column = self.config.review_column,
            label_column = self.config.label_column,
            id_column = self.config.id_column,
            istesting = self.config.istesting,
            embeddingtype = self.config.embeddingtype)
        
    def create_test_train_dr(self,filepath,trainfile,testfile,validatefile):
        train_ds, test_ds,validate_ds = (self.read(filepath / fname) for fname in [trainfile, 
                                                                               testfile,validatefile]) 
        return train_ds,test_ds,validate_ds
    
    def define_vocab(self,datareader):
        vocab = Vocabulary.from_instances(datareader, max_vocab_size=self.config.max_vocab_size)
        return vocab
    
    def define_iterator(self,sort_tuple):
        vocab = Vocabulary()
        iterator = BucketIterator(batch_size=self.config.batch_size, biggest_batch_first=True,
                          sorting_keys=[sort_tuple],
                         )
        iterator.index_with(vocab)
        return iterator
    
    def define_elmo_embedding(self,options,weights):
        #options_file = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_options.json'
        #weight_file = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5'
        elmo_embedder = ElmoTokenEmbedder(options, weights)
        word_embeddings = BasicTextFieldEmbedder({"tokens": elmo_embedder})
        return word_embeddings
    
    def define_bert_embedding(self,pretrained_model="bert-base-uncased"):
        bert_embedder = PretrainedBertEmbedder(pretrained_model=pretrained_model,
                                               top_layer_only=True,)
        word_embeddings = BasicTextFieldEmbedder({"tokens": bert_embedder},
                                                           allow_unmatched_keys = True)
        return word_embeddings
    
    def define_elmo_encoder(self,word_embeddings):
        
        encoder = PytorchSeq2VecWrapper(nn.LSTM(word_embeddings.get_output_dim(), 
                                                self.config.hidden_size, 
                                                bidirectional=self.config.bidirectional, 
                                                batch_first=self.config.batch_first))
        return encoder
    
    def define_bert_encoder(self,bert_dim):
        vocab = Vocabulary()
        bert_dim = bert_dim
        encoder = BertSentencePooler(vocab)
        return encoder
    
     
    
    

In [0]:
class SentimentModel(Model):
    def __init__(self, word_embeddings,encoder,out_sz):
        vocab = Vocabulary()
        
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens,
                id, label):
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

In [0]:
train.to_csv('train.csv',index=False)
validate.to_csv('validate.csv',index=False)
test['sentiment_0']=0
test['sentiment_1']=0
test['sentiment_2']=0
test.to_csv('test.csv',index=False)
sample.to_csv('sample.csv',index=False)

In [0]:
ds = data_setup(config)
filepath = Path("./")
trainfile = 'train.csv'
testfile = 'test.csv'
validatefile = 'validate.csv'
#options = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_options.json'
#weight = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5'
options = '/content/gdrive/My Drive/Studies/innoplexus/elmo_2x4096_512_2048cnn_2xhighway_options.json'
weight = '/content/gdrive/My Drive/Studies/innoplexus/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5'
output_class = 3
sort_tuple = ("tokens","num_tokens")

train_ds,test_ds,validate_ds = ds.create_test_train_dr(filepath,trainfile,testfile,validatefile)

iterator = ds.define_iterator(sort_tuple)
if config.embeddingtype == 'elmo':
    elmo_word_embedding = ds.define_elmo_embedding(options,weight)
    encoder = ds.define_elmo_encoder(elmo_word_embedding)
    
    model = SentimentModel(
        elmo_word_embedding, 
        encoder, 
        output_class
    )

elif config.embeddingtype == 'bert':
    bert_word_embedding = ds.define_bert_embedding()
    bert_dim = bert_word_embedding.get_output_dim()
    encoder = ds.define_bert_encoder(bert_dim)



    model = SentimentModel(
        bert_word_embedding, 
        encoder, 
        output_class
        )
else:
    print("please define embedding type")



#batch = nn_util.move_to_device(batch, 0 if USE_GPU else -1)

4224it [00:22, 183.85it/s]
2924it [00:16, 175.01it/s]
1055it [00:05, 190.18it/s]


In [0]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95)
#momscheduler  = CosineWithRestarts(optimizer,t_initial=4)
#momsche=inverted_triangular(optimizer,cool_down = 10,warm_up = 15)
#momscheduler  = NoamLR(optimizer,model_size=config.hidden_size,warmup_steps=20,factor=0.01)

In [0]:
if USE_GPU: 
    model.cuda() 
else: model
#optimizer = optim.Adam(model.parameters(), lr=config.lr)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95)

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    validation_dataset = validate_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
 #   learning_rate_scheduler = momscheduler,
    shuffle=True,
    patience = 5
    
)

In [0]:
metrics = trainer.train()

loss: 0.4635 ||: 100%|██████████| 132/132 [02:52<00:00,  1.70s/it]
loss: 0.4723 ||: 100%|██████████| 33/33 [00:41<00:00,  2.43s/it]
loss: 0.4473 ||: 100%|██████████| 132/132 [02:45<00:00,  1.12s/it]
loss: 0.4648 ||: 100%|██████████| 33/33 [00:40<00:00,  2.44s/it]
loss: 0.4397 ||: 100%|██████████| 132/132 [02:46<00:00,  1.79s/it]
loss: 0.4643 ||: 100%|██████████| 33/33 [00:40<00:00,  2.44s/it]
loss: 0.4336 ||: 100%|██████████| 132/132 [02:45<00:00,  1.56s/it]
loss: 0.4620 ||: 100%|██████████| 33/33 [00:40<00:00,  2.44s/it]
loss: 0.4196 ||: 100%|██████████| 132/132 [02:45<00:00,  1.16s/it]
loss: 0.4641 ||: 100%|██████████| 33/33 [00:39<00:00,  2.43s/it]
loss: 0.4148 ||: 100%|██████████| 132/132 [02:45<00:00,  1.03s/it]
loss: 0.4699 ||: 100%|██████████| 33/33 [00:40<00:00,  2.44s/it]
loss: 0.4086 ||: 100%|██████████| 132/132 [02:46<00:00,  1.38it/s]
loss: 0.4741 ||: 100%|██████████| 33/33 [00:39<00:00,  2.44s/it]
loss: 0.3974 ||: 100%|██████████| 132/132 [02:46<00:00,  1.08s/it]
loss: 0.4

In [0]:
from allennlp.data.iterators import DataIterator
from tqdm import tqdm
from scipy.special import expit # the sigmoid function

def tonp(tsr): return tsr.detach().cpu().numpy()

class Predictor:
    def __init__(self, model: Model, iterator: DataIterator,
                 cuda_device: int=-1) -> None:
        self.model = model
        self.iterator = iterator
        self.cuda_device = cuda_device
        
    def _extract_data(self, batch) -> np.ndarray:
        out_dict = self.model(**batch)
        return expit(tonp(out_dict["class_logits"]))
    
    def predict(self, ds: Iterable[Instance]) -> np.ndarray:
        pred_generator = self.iterator(ds, num_epochs=1, shuffle=False)
        self.model.eval()
        pred_generator_tqdm = tqdm(pred_generator,
                                   total=self.iterator.get_num_batches(ds))
        preds = []
        with torch.no_grad():
            for batch in pred_generator_tqdm:
                batch = nn_util.move_to_device(batch, self.cuda_device)
                preds.append(self._extract_data(batch))
        return np.concatenate(preds, axis=0)

In [0]:

from allennlp.data.iterators import BasicIterator
# iterate over the dataset without changing its order
seq_iterator = BasicIterator(batch_size=64)
vocab = Vocabulary()
seq_iterator.index_with(vocab)

In [0]:
predictor = Predictor(model, seq_iterator, cuda_device=0 if USE_GPU else -1)
#train_preds = predictor.predict(train_ds) 
test_preds = predictor.predict(test_ds)

100%|██████████| 46/46 [02:39<00:00,  3.33s/it]


In [0]:
t1 = test_preds.argmax(axis=1)
t1 = pd.DataFrame(t1)
#t1.value_counts()
t1.columns =['res']

t = pd.concat([test,t1],axis=1)
t.to_csv('test_pred.csv',index=False)

In [0]:
t.shape

(2924, 8)

In [0]:
sample.shape

(2924, 2)

In [0]:
sample.head()

,unique_hash,sentiment
0,9e9a8166b84114aca147bf409f6f956635034c08,0
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,0
2,50b6d851bcff4f35afe354937949e9948975adf7,0
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,0
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,0


In [0]:
t = t[['unique_hash','res']]

In [0]:
t.columns=['unique_hash','sentiment']

In [0]:
t

,unique_hash,sentiment
0,9e9a8166b84114aca147bf409f6f956635034c08,2
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,2
2,50b6d851bcff4f35afe354937949e9948975adf7,2
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,2
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,2
5,b1950d27d94ceff4e9bf8c7d1fd4b11b35ede4d7,2
6,abafc5b6c5aac6f777cf265e5c7dd80fb793e6bc,2
7,e5550693e72a8335d723ca5fc64da91e1256fb0b,2
8,ee8c500f6402331ff12b0b29d943b6d1699a0b8d,2
9,d261600ba4fc022fac12748845deed56822ff195,2


In [0]:
t['sentiment'].value_counts()

2    2872
1      31
0      21
Name: sentiment, dtype: int64

In [0]:
t.to_csv('test_pred.csv',index=False)
t.to_csv('/content/gdrive/My Drive/Studies/innoplexus/test_pred.csv',index=False)

In [0]:
nnjn 

NameError: ignored